# Tarea 2: utilizar SageMaker Experiments

En este laboratorio, configure un experimento con Amazon SageMaker Experiments. Entrene un modelo de machine learning (ML) con XGBoost, adapte los hiperparámetros para probar varias configuraciones de hiperparámetros y producir un modelo más exacto, y evalúe el rendimiento de su modelo.

## Tarea 2.1: configuración del entorno

Antes de empezar a entrenar su modelo, instale todas las dependencias necesarias.


Consulte [Administre el machine learning con Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html#experiments-features) para obtener más información sobre las funciones de SageMaker Experiments.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.experiments.run import Run, load_run
#from sagemaker.utils import unique_name_from_base  #could be used instead of the date-time append approach, to create a unique Experiment name.
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlasms'

A continuación, importe el conjunto de datos.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

Divida el conjunto de datos en conjuntos de datos de entrenamiento (70 %), validación (20 %) y prueba (10 %). Los conjuntos de datos de entrenamiento y validación se usan durante el entrenamiento. El conjunto de datos de prueba se usa en la evaluación del modelo después de la implementación.

Para realizar el entrenamiento con SageMaker, debe convertir los conjuntos de datos al formato libSVM o CSV. En este laboratorio, se usa el formato CSV para realizar el entrenamiento. 

Consulte [Algoritmo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) para obtener información sobre el algoritmo XGBoost. 
Consulte [Interfaz de entrada/salida para el algoritmo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost) para obtener más información sobre la interfaz de entrada/salida para el algoritmo XGBoost.


In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

Creó dos archivos de conjuntos de datos llamados *train_data.csv* y *validation_data.csv*. 
Cargue estos dos archivos a Amazon Simple Storage Service (Amazon S3).

In [ ]:
#upload-dataset
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## Tarea 2.2: crear un experimento y ejecutar un trabajo de entrenamiento inicial

Use SageMaker Experiments para organizar, comparar y evaluar los experimentos del entrenamiento de modelos de ML, y para hacerles un seguimiento, a través de diversos componentes de entrenamiento. Consulte [SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) para obtener más información sobre SageMaker Experiments. En SageMaker Experiments, estos componentes incluyen conjuntos de datos, algoritmos, hiperparámetros y métricas. 

En esta tarea, complete lo siguiente:
- Cree el experimento en Amazon SageMaker Studio y hágale un seguimiento.
- Cree una ejecución para hacer un seguimiento de las entradas, los parámetros y las métricas.

Primero, cree un nombre para el experimento y asígnele una descripción.

In [ ]:
#create unique experiment name
create_date = strftime("%m%d%H%M")

lab_6_experiment_name = "lab-6-{}".format(create_date)
description = "Using SageMaker Experiments with the Adult dataset."




Luego, defina los valores opcionales para el nombre de una ejecución y las etiquetas.

In [ ]:
# create initial run_name
run_name = "lab-6-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-6', 'Value': 'lab-6-run'}]

print(f"Experiment name - {lab_6_experiment_name},  run name - {run_name}")

## Tarea 2.3: entrenar y adaptar el modelo con el algoritmo XGBoost

El experimento está configurado y listo para el entrenamiento. Tras completar el entrenamiento, puede analizar los resultados en SageMaker Studio. En esta tarea, haga lo siguiente: 

- Entrene el modelo de XGBoost.
- Analice los experimentos en SageMaker Studio.
- Adapte el modelo con hiperparámetros.
- Analice los resultados del ajuste en SageMaker Studio.

### Tarea 2.3.1 entrenar el modelo de XGBoost

Ahora, entrene el modelo con el algoritmo XGBoost y el experimento que creó. 

Los hiperparámetros que configurará serán los siguientes:
- **eta**: disminución del tamaño del paso que se usa en las actualizaciones para evitar el sobreajuste. Después de cada paso de potenciación, puede obtener las ponderaciones de las nuevas funciones directamente. 
- **gamma**: se requiere la reducción de la pérdida mínima para crear una partición más en un nodo hoja del árbol. Cuanto más grande es, más conservador es el algoritmo.
El parámetro eta disminuye las ponderaciones de las funciones para que el proceso de potenciación sea más conservador.
- **max_depth**: profundidad máxima de un árbol. Si se aumenta este valor, el modelo es más complejo y, es probable que, se sobreajuste.
- **min_child_weight**: suma mínima de la ponderación de instancias (hessiana) que se necesita en un proceso secundario. Si el paso de partición del árbol genera un nodo hoja cuya suma de la ponderación de la instancia es inferior a min_child_weight, el proceso de compilación deja de crear más particiones. En los modelos de regresión lineal, esto corresponde a un número mínimo de instancias que se necesitan en cada nodo. Cuanto más grande es el algoritmo, más conservador es.
- **num_round**: la cantidad de rondas (árboles) que se usan para la potenciación. Aumentar los árboles puede aumentar la exactitud del modelo, pero también el riesgo de sobreajustes.
- **objective**: especifica la tarea de aprendizaje y el objetivo de aprendizaje correspondiente.
- **subsample**: relación de submuestras de la instancia de entrenamiento. Cuando se establece en 0,5 XGBoost recopila de forma aleatoria la mitad de las instancias de datos para crear arboles. Esto evita el sobreajuste.
- **verbosity**: verbosidad de los mensajes impresos. Los valores válidos son 0 (silencioso), 1 (advertencia), 2 (información) y 3 (depuración).

La ejecución de este entrenamiento tarda aproximadamente entre 3 y 4 minutos.

Consulte [hiperparámetros](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) para obtener más información sobre los hiperparámetros de XGBoost.

In [ ]:
from sagemaker import image_uris
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')

# initialize hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8
verbosity=0

hyperparameters = {
        "max_depth":max_depth,
        "eta":eta,
        "gamma":gamma,
        "min_child_weight":min_child_weight,
        "subsample":subsample,
        "verbosity":verbosity,
        "objective":objective,
        "num_round":num_round
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    EnableSageMakerMetricsTimeSeries=True,
    hyperparameters=hyperparameters,
    tags = run_tags
)


#Run the training job link to Experiment.
with Run(
    experiment_name=lab_6_experiment_name,
    run_name=run_name,
    tags=run_tags,
    sagemaker_session=sagemaker_session,
) as run:

    run.log_parameters({
                        "eta": eta, 
                        "gamma": gamma, 
                        "max_depth": max_depth,
                        "min_child_weight": min_child_weight,
                        "num_round": num_round,
                        "objective": objective,
                        "subsample": subsample,
                        "verbosity": verbosity
                       })
    
#    you may also specify metrics to log
#    run.log_metric(name="", value=x)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    ) 

### Tarea 2.3.2: evaluar el rendimiento del modelo antes del ajuste

En SageMaker Studio, puede crear gráficos para evaluar sus trabajos de entrenamiento. Por ejemplo, después de ejecutar experimentos en el laboratorio 6, puede revisar el valor validation:logloss_max en formato de gráfico.

En este laboratorio, puede trazar métricas adicionales insertadas justo en el cuaderno.

In [ ]:
#visualize-training-results-table
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
plt.show()

### Tarea 2.3: ajustar el modelo con hiperparámetros

Realizó correctamente el entrenamiento del modelo con SageMaker Experiments. Mientras realiza el entrenamiento, también puede configurar SageMaker a fin de usar hiperparámetros para influir significativamente en el rendimiento del modelo entrenado. SageMaker Studio incluye varias opciones de ajuste de hiperparámetros comunes para el entrenamiento de modelos. Si bien la eficacia de las pruebas de numerosos parámetros puede variar según el conjunto de datos que se use, es posible que también se necesite mucho tiempo y esfuerzo para crear el mejor modelo.

El ajuste de modelos automáticos de SageMaker automatiza la selección de hiperparámetros para optimizar el entrenamiento. Consulte [ajuste automático de modelos](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html) para obtener más información sobre el ajuste automático de modelos. Para usarlo, especifique un rango o una lista de posibles valores para cada hiperparámetro que desee ajustar. El ajuste automático de modelos de SageMaker ejecuta automáticamente varios trabajos de entrenamiento con diversas configuraciones de hiperparámetros. Luego, evalúa los resultados de cada trabajo según una métrica objetiva especificada y selecciona las configuraciones de hiperparámetros para intentos futuros basados en resultados previos. Para cada trabajo de ajuste, usted especifica un número máximo de trabajos de entrenamiento, y el ajuste se completa cuando se alcance ese número.

Los rangos de hiperparámetros que debe configurar serán los siguientes:
- **alpha**: duración de la regularización de nivel 1 en ponderaciones. Si se aumenta este valor, los modelos son más conservadores.
- **eta**: disminución del tamaño del paso que se usa en las actualizaciones para evitar el sobreajuste. Después de cada paso de potenciación, puede obtener las ponderaciones de las nuevas funciones directamente. El parámetro eta disminuye las ponderaciones de las funciones para que el proceso de potenciación sea más conservador.
- **max_depth**: profundidad máxima de un árbol. Si se aumenta este valor, el modelo es más complejo y, es probable que, se sobreajuste.
- **min_child_weight**: suma mínima de la ponderación de instancias (hessiana) que se necesita en un proceso secundario. Si el paso de partición del árbol genera un nodo hoja cuya suma de la ponderación de la instancia es inferior a min_child_weight, el proceso de compilación deja de crear más particiones. En los modelos de regresión lineal, esto corresponde a un número mínimo de instancias que se necesitan en cada nodo. Cuanto más grande es el algoritmo, más conservador es.
- **num_round**: la cantidad de rondas (árboles) que se usan para la potenciación. Aumentar los árboles puede aumentar la exactitud del modelo, pero también el riesgo de sobreajustes.

El ajuste demora aproximadamente 5 minutos en completarse.

Consulte [rangos de hiperparámetros](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) para obtener más información sobre los hiperparámetros de xgboost.

In [ ]:
#tune-model
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# Setup the hyperparameter ranges
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 2),
    'eta': ContinuousParameter(0, 1),
    'max_depth': IntegerParameter(1, 10),
    'min_child_weight': ContinuousParameter(1, 10),
    'num_round': IntegerParameter(100, 1000)
}
# Define the target metric and the objective type (max/min)
objective_metric_name = 'validation:auc'
objective_type='Maximize'
# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type='Auto',
)

with load_run(sagemaker_session=sagemaker_session, experiment_name=lab_6_experiment_name, run_name=run_name) as run:
# Tune the model
    tuner.fit(
        inputs = data_inputs,
        job_name = lab_6_experiment_name,
    )
    run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
 
plt.show()
    

### Tarea 2.3.4: evaluar el rendimiento del modelo después del ajuste

En SageMaker Studio, también puede crear gráficos para evaluar sus trabajos de ajuste. Por ejemplo, después de ejecutar el trabajo de entrenamiento de pruebas del laboratorio 6, puede observar su valor objetivo, **validation:auc_max**, en forma de gráfico.

![Una imagen de los gráficos de validation:error_max en SageMaker Studio.](Task_2_3_4.png)

En este laboratorio, visualice los resultados del mejor trabajo de ajuste usando gráficos en el cuaderno.

In [ ]:
#get_experiment_analytics 
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    sagemaker_session=Session(sess, sm),
)

run_component_analytics.dataframe()

In [ ]:
#visualize-tuning-results-auc-max
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    run_component_analytics.dataframe()["validation:auc - Max"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
else:
    run_component_analytics.dataframe()["validation:auc - Last"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
    
plt.show()

In [ ]:
#visualize-tuning-results-auc-max-scatter
N = 12
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Max"];
else:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Last"];
y = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["num_round"]

plt.scatter(x, y, alpha=0.5)
plt.title("auc_max by num_round")
plt.xlabel("validation:auc - Max")
plt.ylabel("num_round");
plt.show()

Por último, puede imprimir el mejor trabajo de ajuste según su métrica objetiva.

In [ ]:
#print-best
tuner.best_training_job()

### Tarea 2.3.5: graficar las métricas del experimento con funciones incorporadas de SageMaker Studio

El método mencionado anteriormente crea gráficos a partir de métricas del experimento mediante el uso de celdas insertadas del cuaderno. Una opción diferente es trazar algunas de las métricas del experimento usando funciones dentro de SageMaker Studio. Ahora que el experimento se ejecutó al menos una vez, cree un nuevo gráfico de barras en SageMaker Studio.

La próxima tarea abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_6.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **lab_6.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora, puede ver las instrucciones mientras explora los artefactos.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar los artefactos, seleccione la pestaña **lab_6.ipynb** para volver al cuaderno.

1. Seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
1. Ejecute **Experiments** (Experimentos).

Verá la pestaña **Experiments** (Experimentos) en SageMaker Studio.

1. Seleccione el experimento que comienza con *lab-6-*.

SageMaker Studio muestra la lista de **ejecuciones** incluidas en ese experimento.

1. Seleccione la opción en la columna **Name** (Nombre) junto a todas las ejecuciones disponibles que están asociadas con el trabajo de ajuste de hiperparámetros.
1. Desactive la 
1. Elija <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Analyze</span> (Analizar).

En SageMaker Studio, verá la pestaña **Run Analyze Chart** (Ejecutar gráfico de análisis).

1. En la mitad inferior de la pestaña, seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">+ Add Chart</span> (+ Agregar gráfico) en la sección de gráficos.
1. Elija **Bar** (Barras).

Verá la pestaña **Add Chart** (Agregar gráfico) en SageMaker Studio.

1. Para **Y-axis** (eje Y), seleccione **min_child_weight**.
1. Elija <span style="background-color:#73cdf9; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-radius:2px; border-width:3px; margin-right:5px; white-space:nowrap">Create</span> (Crear).

En la sección de gráficos, se guardará un gráfico de barras que muestra *min_child_weight* por *ejecución* en el experimento.

1. Repita este proceso y cree un nuevo gráfico de barras para la métrica **train:auc**.
1. Repita este proceso y cree un nuevo gráfico de barras para la métrica **validation:auc**.

### Conclusión

¡Felicitaciones! Usó SageMaker Experiments para entrenar y ajustar modelos. En el siguiente laboratorio, use SageMaker Debugger para obtener información sobre posibles problemas durante el entrenamiento de un modelo.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.